In [23]:
import my_utils as mu
import pandas as pd
import json

In [4]:
mu.oracle_totalExecute('SELECT COUNT(*) FROM RAWDATA')

oracle open!
oracle close!


,COUNT(*)
0,363520


In [3]:
mu.rawdataIntegrityKeeper()

RawData 테이블의 결성 확인. 삭제를 진행합니다.
대상 game_id: ['KR_6373818275', 'KR_6495670157']


부적절한 매치 데이터 삭제 중...: 100%|██████████| 2/2 [00:00<00:00,  8.43it/s]


In [5]:
rawdata = mu.oracle_totalExecute('SELECT * FROM RAWDATA')

oracle open!
oracle close!


In [7]:
rawdata_dict = rawdata.to_dict()

In [8]:
rawdata_cols = [a for a in rawdata_dict.keys()]

In [11]:
rawdata_cols.insert(5, 'PARTICIPANT_NAME')

In [13]:
newdata_dict = {a:{} for a in rawdata_cols}

In [17]:
rawdata_cols[5]

'PARTICIPANT_NAME'

In [25]:
json.loads(rawdata_dict['SUMMONER'][0])['summonerName']

'쿨감폿'

In [16]:
# for idx in range(len(rawdata_dict['VERSION'])):
#     for col in rawdata_cols:
#         if col not in rawdata_cols[9:]:
#             newdata_dict[col][idx] = rawdata_dict[col][idx]
#         elif col == 'CHAMPION' or col == 'KDA':
#             newdata_dict[col][idx] = rawdata_dict[col][idx].replace("'",'"').replace('True','1').replace('False','0')
#         else:
#             newdata_dict[col][idx] = rawdata_dict[col][idx].replace("'",'"')

In [26]:
for idx in range(len(rawdata_dict['VERSION'])):
    for i, col in enumerate(rawdata_cols):
        if i == 5: newdata_dict[col][idx] = json.loads(rawdata_dict['SUMMONER'][idx])['summonerName']
        else: newdata_dict[col][idx] = rawdata_dict[col][idx]

In [32]:
mu.insertDataFrameIntoTable(pd.DataFrame(newdata_dict), 'RAWDATA_C')

>>> 테이블의 컬럼 정보 검색 중...
>>> 테이블의 Primary Key 정보 검색 중...
>>> 테이블에 데이터 삽입 중...


 37%|███▋      | 136083/363540 [45:46<1:15:55, 49.93it/s]

In [ ]:
mu.oracle_totalExecute('DROP TABLE RAWDATA CASCADE CONSTRAINTS')
mu.oracle_totalExecute("""
    CREATE TABLE RawData
    (
    version            VARCHAR(30)   NOT NULL,
    game_create_time   NUMBER(16)    NOT NULL,
    game_duration      NUMBER(10)    NOT NULL,
    game_id            VARCHAR(16)   NOT NULL,
    participant_number NUMBER(2)     NOT NULL,
    participant_name   NVARCHAR2(20) NOT NULL,
    champion_id        NUMBER(3)     NOT NULL,
    lane               VARCHAR(7)    NOT NULL,
    participant_puuid  VARCHAR(100)  NOT NULL,
    api_key            VARCHAR(50)   NOT NULL,
    game               VARCHAR(600)  NOT NULL,
    summoner           VARCHAR(300)  NOT NULL,
    champion           VARCHAR(250)  NOT NULL,
    spell              VARCHAR(50)   NOT NULL,
    skill              VARCHAR(100)  NOT NULL,
    skillTree          VARCHAR(70)   NOT NULL,
    rune               VARCHAR(300)  NOT NULL,
    item               VARCHAR(150)  NOT NULL,
    startItem          VARCHAR(50)   NOT NULL,
    itemTree           VARCHAR(30)   NOT NULL,
    kda                VARCHAR(200)  NOT NULL,
    gold               VARCHAR(120)  NOT NULL,
    cs                 VARCHAR(70)   NOT NULL,
    turret             VARCHAR(60)   NOT NULL,
    damage             VARCHAR(500)  NOT NULL,
    vision             VARCHAR(100)  NOT NULL,
    CONSTRAINT PK_RawData PRIMARY KEY (version, game_id, participant_number))""")
mu.oracle_totalExecute("""
ALTER TABLE RawData
  ADD CONSTRAINT FK_ApiKey_TO_RawData
    FOREIGN KEY (api_key)
    REFERENCES ApiKey (api_key)""")

In [ ]:
mu.insertDataFrameIntoTable(pd.DataFrame(newdata_dict), 'RAWDATA')